<a href="https://colab.research.google.com/github/thedarredondo/data-science-fundamentals/blob/main/Unit4/Unit4NotesSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import pymc as pm

#Unit 4: Hierachical Models

In this unit we will cover how to model data containing sepreate, but related, groups. We'll start with a continuous data example; here's how we'll break it down. We will:

1. Make a model and assume that the groups are unrelated, or *independent*
2. Alter that model by changing our assumption so that the groups are related, or have some amount of *dependence*
3. Compare and contrast those models using a plot.
4. Cover the same concept with a discrete data set, introducing some complexities and problems with creating groups

Let's get started.

### Loading and Visualizing our Data

The below chemical shift data set is different from that used previous unit; it contains theoreitcal and experimental chemical shift values for 19 different amino acids. Let's view the raw data to see what I mean.

In [ ]:
#load the data from my github page.
#cs stands for chemical shift data
url_cs = 'https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/main/Data/chemical_shifts_theo_exp.csv'
#I could have put the url string directly into pd.read_csv()
cs_data = pd.read_csv(url_cs)

In [ ]:
#print a the begining and end of the raw data set, as a pandas dataframe
cs_data

We are interested in only three columns for right now:
- aa, which stands for amino acid
- theo, which is the theoretical chemical shift value, calculated using "quantum methods", to quote our textbook.
- exp, which is the observed chemical shift value from Nuclear Magnetic Reasonance (NMR)

The following two plots help visualize the theoriteical values for each protein, and the experimental values for each protein.

In [ ]:
#visualize theoretical chem shift values for all the amino acids
sns.violinplot(cs_data, x='theo', y='aa', inner='quart',hue='aa')
sns.despine(top=True, right=True, bottom=True, left=True) #removes border box in plot

In [ ]:
#visualize experimental chem shift values for all the amino acids
sns.violinplot(cs_data, x='exp', y='aa', inner='quart', hue='aa')
sns.despine(top=True, right=True, bottom=True, left=True)

**Task1**:

Without building a model, answer the following:

For each amino acid, are the theoretical chemical shift values different from
the experimental values?

**Answer1**:

[*write your answer here*]

Here's code to find the raw difference between the theoretical and expermental chemical shift values within each amino acid, and to store that difference in our data.

In [ ]:
#element wise subtraction
diff_TminusE = cs_data.theo - cs_data.exp
#adds a column called 'diff_TminusE' as the fifth column of cs_data
cs_data.insert(loc=4,column='diff_TminusE',value=diff_TminusE)

In [ ]:
#print the dataframe with the new column
cs_data

In [ ]:
#visualize difference in chem shift values for all the amino acids
sns.violinplot(cs_data, x='diff_TminusE', y='aa', inner='quart',hue='aa')
sns.despine(top=True, right=True, bottom=True, left=True) #removes border box in plot

**Task2**:

Using the graph of raw differences, answer the following:

- For each amino acid, are the theoretical chemical shift values different from
the experimental values?

- If you had to use one likelihood to model the chemical shift for each amino acid, which one would you use?

**Answer2**:

[*write your answer here*]

Visualizing the raw data and making a guess from it is neccessary, but not sufficient. Was does that mean? We have, at this point, got to the start line.

We understand the information we have as well as we can, and we've used our knowldge to make a preliminary guess: we think the difference between the theoretical chemical shift values and the experimental chemical shift values is around zero. In other words, **we think the "quantum methods" more or less predict the actual chemical shift values in the 19 amino acids we looked at.**

Its the "more or less" part in the last statement that our model will boslter. In the simple models from these first 5 units, its often possible to visualize the raw data like we did and come to a reasonable conclusion. Our models, right now, confirm or deny our intuition, and also give us a mathematical way of saying how confident we should be in our conclusion.

In later units, with more complicated data and goals, our models will end up being the best summarization method. But we should always try to make visualizations without our models first, as we did here.


In order to make a model for the differences between the chemical shift values for each amino acid--and thus confirm our hypothesis--, we need a to give our model a way to know which difference corresponds to which amino acid. The next few code blocks do this.

In [ ]:
#identifies all the different amino acids in our data set
#and stores that information, plus original info, in the variable aa_encode
aa_encode = pd.Categorical(cs_data['aa'])
aa_encode

In [ ]:
#Replaces all the amino acid names with a number in the variable aa_idx
aa_idx = aa_encode.codes
aa_idx

In [ ]:
#stores the category names in a dictionary
#this dictionary format is required
#for pymc to properly read the categories
aa_coords = {"aa": aa_encode.categories}
aa_coords

For now, we'll also make the assumption that all our priors, and our likelihood, are normal/halfnormal. This is for two reasons:
- To keep things simple while we explore hierarchical models
- If all we know about our data is that there is an expected value and variation around that value, then its distribution will be normal. This a good assumption to start our model building process with.

##Continuous Data, Independent Groups

Our data is grouped into 19 different amino acids. To start, I'll assume that knowing soemthing about one amino acid  provides 0 information about any of the other amino acids. This assumptions seems wrong to me, but we'll follow through with assuming amino acids are independent because

- We will contrast this independent model with a partial dependence model, to see how changing independence between amino acid assumption changes our analysis.

**Task3**:

Do you think amino acids are independent from one another, with respect to chemical shift and quantum methods?

**Answer3**:

[*write your answer here*]

Let's see the code for the model where we assume each amino acid is independent from one another.

In [ ]:
with pm.Model(coords=aa_coords) as cs_nh_model:
    #priors
    #dims="aa" means to apply the prior seperately to each amino acid
    μ = pm.Normal('μ', mu=0, sigma=10,
                  dims="aa")
    σ = pm.HalfNormal('σ', sigma=10,
                      dims="aa")

    #The idx variable tells my model which diffs belong to the same amino acid
    #this will cause my model to generate a posterior for each amino acid
    y = pm.Normal('y',
                  mu = μ[aa_idx],
                  sigma = σ[aa_idx],
                  observed=cs_data.diff_TminusE)

    #cs for chemshift, nh for nonheirarchical
    idata_cs_nh = pm.sample()

In [ ]:
#plots the middle 99.7% of each amino acids' posterior distribution of the mean
#why 99.7%? No reason, just seems like a good number to me.
#Feel free to pick your own hdi %; the default is 94%.
axes = az.plot_forest(idata_cs_nh, kind="ridgeplot",
                      var_names='μ',
                      combined=True, figsize=(12, 9), hdi_prob=0.997)

**Task4**:

Using cs_nh_model and the visualization of its posteriors above, answer the following:

Is there a difference between the theoretical and experimental chemical shift values in the amino acids from cs_data?

**Answer4**:

[*write your answer here*]


## Continuous Data, some Dependence among Groups

The model we made and any conclusions we can reach from it are only as good as our assumptions, and we assumed that using one amino acid to predict another was foolish.

We will now relax that assumption, and build a model that automatically uses information about the mean chemical shift of one amino acid to predict the mean of another.

By what black magic is this possible? Before answering that question, and I'll ask you a different question.

**Task5**:

Can a prior have a prior? If so, what would that do, and why would we ever want that?

**Answer5**:

[*write your answer here*]

Here' how to use hyperpriors to let our knowledge of one amino acid's mean chemical shift affect anothers.

In [ ]:
with pm.Model(coords=aa_coords) as cs_h_model:
    # hyperpriors. Notice how there's no dims argument.
    #this means that the hyperprior is shared across all amino acids
    μ_mu = pm.Normal('μ_mu', mu = 0, sigma = 10)
    μ_sd = pm.HalfNormal('μ_sd', sigma = 10)

    # priors. There is a different prior for each amino acid,
    #but now the μ priors 'talk' to one another through μ_mu and μ_sd
    μ = pm.Normal('μ',
                  mu = μ_mu,
                  sigma = μ_sd,
                  dims = "aa")

    σ = pm.HalfNormal('σ', sigma = 10,
                      dims = "aa")

    #likelihood; same as before
    y = pm.Normal('y',
                  mu = μ[aa_idx],
                  sigma = σ[aa_idx],
                  observed = cs_data.diff_TminusE)

    #cs for chemshift, h for heirarchical
    idata_cs_h = pm.sample()

**Task6**:

Considering only the code for cs_h_model and cs_nh_model, write down similarities and differences.

**Answer6**

[*write your answer here*]

Let's see what the hierarchical model cs_h_model's posterior looks like, and compare it to the non-hierarchical model, cs_nh_model.

In [ ]:
#plots a 99.7% hdi for the posteriors of both models on the same plot
axes = az.plot_forest([idata_cs_nh, idata_cs_h], kind="forestplot",
                      model_names=['non_hierarchical', 'hierarchical'],
                      var_names='μ', combined=True, figsize=(12, 10),hdi_prob=0.997)

#plots overall mean of the hierarchical model; do not confuse with 0
y_lims = axes[0].get_ylim()
axes[0].vlines(idata_cs_h.posterior['μ_mu'].mean(), *y_lims, color="k", ls=":");

**Task7**:

Focus on the white dots, which are the means of the posteriors. How did allowing our model to use one amino acid's chemical shift mean to predict another's change our our predictions?

In other words, how do the hierarchical model, cs_h_model, and non-hierarchical model, cs_nh_model, differ?

**Answer7**:

[*write your answer here*]

Below are two visualizations, one of the non-hierarchical model, and one of the hierarchical model. Make sure to use:
```
pm.model_to_graphviz()
```
whenever you have a model more complicated than cs_nh_model

In [ ]:
pm.model_to_graphviz(cs_nh_model)

In [ ]:
#the hyper priors only affect μ, and they are not split up by aa, by amino acid.
pm.model_to_graphviz(cs_h_model)

Notice that there's no hyperpriors for $σ$; this means we assume that the variance of one amino acid is independent from variance of another.

## Discrete Data, Dependent Groups

For our discrete data set, we'll do another sports statistics example, but this time we'll cover soccer (football).

We'll also use a model that doesn't behave well with pymc, but we'll still trust its predictions. We'll dicuss why.

The following code will load the data in, and do the neccessary cleaning. Make sure you're reading my comments--I didn't write them just to help me.

In [ ]:
#load the data from my github page.
#sccr stands for soccer
url_sccr = 'https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/main/Data/football_players.csv'

#Turns the position column into a categorical variable
sccr_data = pd.read_csv(url_sccr,dtype={'position':'category'})

Look at the code below, focusing on the last two columns, goals and shots. This should remind you of the Wembanyama FT example, where we had makes and attempts.
Except there's 2681 players from the Premier League, Ligue 1, Bundesliga, Serie A, and La Liga, instead of 1 NBA player. There's also a categorical variable named position, and these data were complied from 2017-2020.

What's pertinent to use is that we'll use position the same way we used 'aa', amino acid, in the last example.

In [ ]:
sccr_data

**Task8**:

Plot goals and shots from sccr_data. I'd use a violin plot.

Then consider how you would select a prior for goals and shots.

In [ ]:
#ASNWER8


In [ ]:
#ASNWER8 cont.


**Answer8 cont.**:

[*write your answer here*]

Previously, we would have turned goals into an array of $1$s, and [shots - goals] into an array of $0$s. That worked great when we only cared about one player, and its still possible now, but it'd be nice if we could just shove 'goals' and 'shots' into our model as is.

And we can! But there's a price to paid for that decision, this time around . . .

Before we discuss that price, let's review how to get pymc to recognize the grouping we care about, soccer position.

In [ ]:
#this is an array of the soccer positions,
#except they are given an int (number) label instead of a string (of letters)
pos_idx = sccr_data.position.cat.codes.values

#an array of the four strings that represent position
pos_codes = sccr_data.position.cat.categories

#puts coords in a dict, so pymc can read them
sccr_coords = {"pos": pos_codes}

**Task9**:

Explain what the code block above is doing, and why we need to do it before running our pymc model. Make sure to reference the chemical shift amnio acid example.

**Answer9**:

[*write your answer here*]

Run the model below; this model takes about 6-8 minutes to run, and would likely have many divergences if I hadn't changed the target_accept parameter. I changed target_accept from the default of 0.8 to 0.95, based off our textbooks recommendation. Generally, if you can get your model to 0 divergences just based off changing target_accept, then its good enough.

That said, you should probably have some idea why pymc is throwing a fit before you try to press on without changing your model structure. In this case, the main problem is that goal keepers rarely shoot goals, so their shooting percentage is hard to predict.

In [ ]:
with pm.Model(coords = sccr_coords) as sccr_model:
    # Hyper parameters
    #
    μ = pm.Beta('μ', alpha = 2, beta = 20)
    ν = pm.Gamma('ν', mu = 55, sigma = 45)


    # Parameters for positions
    μ_p = pm.Beta('μ_p', mu = μ, nu = ν, dims = "pos")

    ν_p = pm.Gamma('ν_p', mu = 55, sigma = 45, dims="pos")

    # Parameter for players
    θ = pm.Beta('θ', mu = μ_p[pos_idx], nu = ν_p[pos_idx])

    #This tells our model how many shots and goals each player has
    _ = pm.Binomial('gs',
                    n = sccr_data.shots.values, #shots for each player
                    p = θ, #guess of goals per shot
                    observed = sccr_data.goals.values) #goals per player

    #target_accept has a default of 0.8. Increasing it makes us less likely to
    #have divergences
    idata_sccr = pm.sample(target_accept = 0.95)

Below is a visualization of the markov chains that did our inference for us and generated our posterior. A healthy chain is tightly packed, with a steady amplitude.

Notice that μ_p and  ν_p each had an unheathly chain, but that its other chains weren't so bad.
These are the chains for goals and shots for the Goal Keeper (GK) postion; as we'll see below, the model had a hard time predicting GK. That hard time ended up making the chains for μ and ν a little wobbly, and made our prediction for θ more uncertain as well.

In [ ]:
#this gives a visualization of what pymc was doing in those 6 to 8 minutes
#...which also takes 6 to 8 minutes to run
az.plot_trace(idata_sccr)

The following code will give you something called $\hat{R}$ values (pronounced "r hat"); these are how I know the faulty chains are from the GK position.

All you need to know about $\hat{R}$ values is that values between 1 and 1.001 are good, and values above 1.1 are bad.


**Task10**:

Using the values from running rhat function on idata_sccr, explain how I was able to know that the GK group is the problem.

**Answer10**:

[*write your answer here*]

In [ ]:
#Using arviZ to find the rhat values
az.rhat(idata_sccr)

The following will plot the means for each position; you can see that the model has no idea what the shooting percentage (goals/shots) should be for GKs.

Its pretty confident about the other positions though, and its predictions seem to match reality. Forwards (FW) role on a scoccer team is to shoot goals, so it makes sense that they have a higher shooting precentage.

In [ ]:
# plots a 99.8% HDI for the positions shooting percentage means
az.plot_forest(idata_sccr, var_names=['μ_p'], combined=True, figsize=(12, 3), kind = 'ridgeplot', hdi_prob=0.998)

As a final treat, here's a visualization of how Lionel Messi's shooting percentage compares to other FWs, and to other professional soccer players.

In [ ]:
#preps a plot object to have three seperate plots, all with the same x axis
_, ax = plt.subplots(3, 1, figsize=(12, 11), sharex=True)

#plot the overall mean shooting percentage
az.plot_posterior(idata_sccr, var_names='μ', ax=ax[0])
ax[0].set_title(r"Global mean")

#plot the fowrads shooting percentage
az.plot_posterior(idata_sccr.posterior.sel(pos="FW"), var_names='μ_p', ax=ax[1])
ax[1].set_title(r"Forward position mean")

#plot Linonel Messi's shooting pecentage
az.plot_posterior(idata_sccr.posterior.sel(θ_dim_0=1457), var_names='θ', ax=ax[2])
ax[2].set_title(r"Messi mean")

## Summary

This unit covered hierarchical models, and not only showed us that priors can have priors, but that these hyper priors can be useful too.

Hyperpriors were used to allow sub groups within a data set to inform one another; we saw this with amino acids' chemical shift values pulling each other towards a common mean. This made us more likely to conclude that a given amino acid was well predicted by theoretical methods.

We then finished by creating a model for professional soccer player shooting percentage. This model introduced us to some new model checking tools, and helped us learn that breaking our data into subgroups can make pymc's job harder, even if these group labels reflect reality.

All the models in this unit were more complicated than we've seen before, and they demanded that we study the structure of our raw data.To do this, we used soem new graphical tools.

In this unit's project, you'll work on example similar to the soccer data set, so that you can get more familiar with breaking a data set into groups, and then assigning priors to those groups. With that, you'll have the foundation for how to encode most context and assumptions into a model.

